# Disregard This:  Read Files How To
## With Spark, you can read files directly from GDELT S3

I pulled down a single day's files into an S3 bucket
* aws s3 cp s3://gdelt-open-data/v2/events/ s3://labadie.gdelt/ --recursive --exclude "*" --include "20180814??????.export.csv"
* I don't think you can use wildcards with hadoop distcp

I then brought those into hdfs
  * hadoop distcp s3://labadie.gdelt/ gdelt/

# Initiate Spark

In [3]:
import findspark
findspark.init()

from pyspark import SparkContext  
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BDAproject").getOrCreate()

sc = spark.sparkContext # Link SparkContext to SparkSession

# Get GDELT Events Schema

In [5]:
import urllib.request  # lib that handles URLs
#import requests
import io
import pandas as pd

target_url="https://raw.githubusercontent.com/linwoodc3/gdelt2HeaderRows/master/schema_csvs/GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv"
data = urllib.request.urlopen(target_url).read().decode('utf8')
formats = pd.read_csv(io.StringIO(data))
formats["dataType"].replace({"INTEGER":"IntegerType",
                             "FLOAT":"LongType",
                             "STRING":"StringType"},inplace=True)

### Build the schema

In [6]:
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType

gdeltschema = []
for row in formats.iterrows():
    sf = StructField(row[1]["tableId"],
                     StringType(),
                     #globals()[row[1]["dataType"]](),
                     True)
    gdeltschema.append(sf)
    
gdeltschema=StructType(gdeltschema)

## Read in Data

In [11]:
df = spark.read.format("csv") \
    .option("header", "false") \
    .option("delimiter","\t") \
    .schema(gdeltschema) \
    .load("s3://gdelt-open-data/v2/events/20180814*.export.csv") # Note just one day:  20180814
df.createOrReplaceTempView("gdelt_df")
df.cache()

DataFrame[GLOBALEVENTID: string, SQLDATE: string, MonthYear: string, Year: string, FractionDate: string, Actor1Code: string, Actor1Name: string, Actor1CountryCode: string, Actor1KnownGroupCode: string, Actor1EthnicCode: string, Actor1Religion1Code: string, Actor1Religion2Code: string, Actor1Type1Code: string, Actor1Type2Code: string, Actor1Type3Code: string, Actor2Code: string, Actor2Name: string, Actor2CountryCode: string, Actor2KnownGroupCode: string, Actor2EthnicCode: string, Actor2Religion1Code: string, Actor2Religion2Code: string, Actor2Type1Code: string, Actor2Type2Code: string, Actor2Type3Code: string, IsRootEvent: string, EventCode: string, EventBaseCode: string, EventRootCode: string, QuadClass: string, GoldsteinScale: string, NumMentions: string, NumSources: string, NumArticles: string, AvgTone: string, Actor1Geo_Type: string, Actor1Geo_FullName: string, Actor1Geo_CountryCode: string, Actor1Geo_ADM1Code: string, Actor1Geo_ADM2Code: string, Actor1Geo_Lat: string, Actor1Geo_Lon

In [12]:
df.count()

178416

In [13]:
df.show(5)

+-------------+--------+---------+----+------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+-------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+-------------+-------------+---------+--------------+-----------+----------+-----------+-----------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+
|GLOBALEVENTI

## Try some SQL and pyspark queries

GLOBALEVENTID| SQLDATE|MonthYear|Year|FractionDate|Actor1Code|Actor1Name|Actor1CountryCode|Actor1KnownGroupCode|
Actor1EthnicCode|Actor1Religion1Code|Actor1Religion2Code|Actor1Type1Code|Actor1Type2Code|Actor1Type3Code|Actor2Code|
Actor2Name|Actor2CountryCode|Actor2KnownGroupCode|Actor2EthnicCode|Actor2Religion1Code|Actor2Religion2Code|
Actor2Type1Code|Actor2Type2Code|Actor2Type3Code|IsRootEvent|EventCode|EventBaseCode|EventRootCode|QuadClass|
GoldsteinScale|NumMentions|NumSources|NumArticles| AvgTone|Actor1Geo_Type|  Actor1Geo_FullName|Actor1Geo_CountryCode|
Actor1Geo_ADM1Code|Actor1Geo_ADM2Code|Actor1Geo_Lat|Actor1Geo_Long|Actor1Geo_FeatureID|Actor2Geo_Type|  
Actor2Geo_FullName|Actor2Geo_CountryCode|Actor2Geo_ADM1Code|Actor2Geo_ADM2Code|Actor2Geo_Lat|Actor2Geo_Long|
Actor2Geo_FeatureID|ActionGeo_Type|  ActionGeo_FullName|ActionGeo_CountryCode|ActionGeo_ADM1Code|ActionGeo_ADM2Code|
ActionGeo_Lat|ActionGeo_Long|ActionGeo_FeatureID| DATEADDED| SOURCEURL

In [44]:
from pyspark.sql.functions import countDistinct
from pyspark.sql import functions as F

# Min, Max, Avg, and distinct Count of NumMentions
df.agg(F.min(df.NumMentions),F.max(df.NumMentions),F.avg(df.NumMentions), F.countDistinct(df.NumMentions)).show()

+----------------+----------------+-----------------+---------------------------+
|min(NumMentions)|max(NumMentions)| avg(NumMentions)|count(DISTINCT NumMentions)|
+----------------+----------------+-----------------+---------------------------+
|               1|              99|5.064181463545871|                        108|
+----------------+----------------+-----------------+---------------------------+



In [45]:
# Distinct Root Events by month
df.groupBy(col("MonthYear")).agg(countDistinct(col("IsRootEvent"))).show()

+---------+---------------------------+
|MonthYear|count(DISTINCT IsRootEvent)|
+---------+---------------------------+
|   201808|                          2|
|   200808|                          2|
|   201807|                          2|
|   201708|                          2|
+---------+---------------------------+



In [48]:
# Distinct Mentions by month
df.groupBy(col("MonthYear")).agg(countDistinct(col("NumMentions"))).show()

+---------+---------------------------+
|MonthYear|count(DISTINCT NumMentions)|
+---------+---------------------------+
|   201808|                        108|
|   200808|                          7|
|   201807|                         21|
|   201708|                         28|
+---------+---------------------------+



In [50]:
# Distinct Mentions by Actor1Name
df.groupBy(col("Actor1Name")).agg(countDistinct(col("NumMentions"))).show(10)

+--------------------+---------------------------+
|          Actor1Name|count(DISTINCT NumMentions)|
+--------------------+---------------------------+
|      LLOYDS BANKING|                          1|
|               SAVOY|                          3|
|                A US|                          8|
|           TRAVELLER|                          9|
|              SOMALI|                          9|
|              HUNTER|                         12|
|             ARMENIA|                         11|
|MINIST OF GOVERNMENT|                          1|
|               OSAGE|                          3|
|  ANADARKO PETROLEUM|                          2|
+--------------------+---------------------------+
only showing top 10 rows



In [51]:
# Mentions by Actor1Name - slightly different from distinct count for some actors
df.groupBy(col("Actor1Name")).count().show(10)

+--------------------+-----+
|          Actor1Name|count|
+--------------------+-----+
|              HUNTER|   61|
|           TRAVELLER|   40|
|              SOMALI|   37|
|                A US|   67|
|             ARMENIA|  120|
|MINIST OF GOVERNMENT|    1|
|  ANADARKO PETROLEUM|    4|
|             GWALIOR|    1|
|               SAVOY|    8|
|               OSAGE|    3|
+--------------------+-----+
only showing top 10 rows



In [74]:
# Group by and sort by number of Mentions and display count of how many times that number of mentions appears
from pyspark.sql.functions import desc

df.groupBy(col("NumMentions")).count().sort("NumMentions").show()

+-----------+-----+
|NumMentions|count|
+-----------+-----+
|          1|23902|
|         10|26122|
|        100|    9|
|        102|    2|
|        104|    3|
|         11|  103|
|        110|    9|
|        115|    1|
|        116|    1|
|        117|    1|
|        119|    1|
|         12| 1272|
|        120|   13|
|        124|    1|
|        125|    1|
|        126|    1|
|        128|    1|
|         13|   96|
|        130|    4|
|        135|    1|
+-----------+-----+
only showing top 20 rows



In [83]:
# Group by and sort by number of Mentions and display count of how many times that number of mentions appears
# The orderBy descending doesn't work right - ignores 100s because they start with 1.
from pyspark.sql.functions import desc

df.groupBy(col("NumMentions")).count().orderBy(col("NumMentions").desc()).show(10)

+-----------+-----+
|NumMentions|count|
+-----------+-----+
|         99|    1|
|         98|    2|
|         96|   11|
|         92|    6|
|         90|   13|
|          9|  491|
|         88|    1|
|         86|    2|
|         84|    9|
|         83|    1|
+-----------+-----+
only showing top 10 rows



In [87]:
from pyspark.sql.functions import desc

df.orderBy(col("DATEADDED").desc()).show(2)

+-------------+--------+---------+----+------------+----------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+--------------------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+-------------+-------------+---------+--------------+-----------+----------+-----------+-----------------+--------------+------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+---------------------+------------------+------------------+-------------+--------------+-------------------+--------------+--------------------+
|GLOBALE

#### Using Spark.SQL

In [ ]:
# gdelt_df is registered table name

In [32]:
# Count number of observations
spark.sql("select count(*) as obs from gdelt_df").show()

+------+
|   obs|
+------+
|178416|
+------+



In [43]:
# Three ways to do the same thing - count number of times Actor1Name = QUEENSLAND
from pyspark.sql.functions import col

df.filter(col("Actor1Name")=='QUEENSLAND').count()  # Use DataFrame with columns method and pyspark col function
df.filter(df.Actor1Name=="QUEENSLAND").count()  # Use DataFrame columns method
df.filter("Actor1Name like 'QUEENSLAND'").count()  # Plain SQL

63

In [98]:
# Number of events by year, sorted by year
spark.sql('SELECT Year, COUNT(GLOBALEVENTID) AS num_events FROM gdelt_df GROUP BY Year ORDER BY Year ASC').show()

+----+----------+
|Year|num_events|
+----+----------+
|2008|        23|
|2017|      1672|
|2018|    176721|
+----+----------+



In [99]:
# Number of events by event code, sorted by number of events
spark.sql('SELECT EventCode, COUNT(GLOBALEVENTID) AS num_events FROM gdelt_df GROUP BY EventCode ORDER BY num_events DESC').show()

+---------+----------+
|EventCode|num_events|
+---------+----------+
|      010|     14160|
|      042|     13819|
|      043|     12901|
|      020|     10313|
|      051|      9838|
|      040|      9826|
|      190|      7515|
|      036|      7056|
|      173|      6433|
|      046|      5319|
|      112|      3991|
|      090|      3726|
|      120|      3450|
|      013|      3363|
|      111|      3339|
|      110|      3220|
|      012|      3056|
|      071|      3047|
|      030|      2813|
|      060|      2702|
+---------+----------+
only showing top 20 rows



In [110]:
# Number of events where actor1name and actor2name are specific values (watch case)
spark.sql("""
    SELECT Year, EventCode, Actor1Name, Actor2Name, COUNT(GLOBALEVENTID) AS num_events 
    FROM gdelt_df 
    WHERE Actor1Name='ARMENIA' and Actor2Name='RUSSIA'
    GROUP BY Year, EventCode, Actor1Name, Actor2Name
    ORDER BY num_events DESC
    """).show()

+----+---------+----------+----------+----------+
|Year|EventCode|Actor1Name|Actor2Name|num_events|
+----+---------+----------+----------+----------+
|2018|     1246|   ARMENIA|    RUSSIA|         2|
+----+---------+----------+----------+----------+



In [111]:
# Number of events where actor1name and actor2name can be fuzzy values (watch case)
spark.sql("""
    SELECT Year, EventCode, Actor1Name, Actor2Name, COUNT(GLOBALEVENTID) AS num_events 
    FROM gdelt_df 
    WHERE Actor1Name='ARMENIA' and Actor2Name LIKE 'RUSSIA%'
    GROUP BY Year, EventCode, Actor1Name, Actor2Name
    ORDER BY num_events DESC
    """).show()

+----+---------+----------+----------+----------+
|Year|EventCode|Actor1Name|Actor2Name|num_events|
+----+---------+----------+----------+----------+
|2018|     1246|   ARMENIA|    RUSSIA|         2|
|2018|      141|   ARMENIA|   RUSSIAN|         2|
|2018|      043|   ARMENIA|   RUSSIAN|         1|
+----+---------+----------+----------+----------+



#### Extra Code Bits

In [ ]:
#### List of columns subset to return
GLOBALEVENTID, SQLDATE, MonthYear, Year, Actor1Name, Actor2Name, IsRootEvent, EventCode, NumMentions, 
NumSources, NumArticles, AvgTone, Actor1Geo_ADM1Code, Actor2Geo_ADM1Code, ActionGeo_FullName, 
ActionGeo_CountryCode, ActionGeo_ADM1Code, DATEADDED, SOURCEURL

In [ ]:
# Number and avg tone of mentions from the Brexit example
# This uses the mentions files

create external table mentions (
globaleventid INT,
eventtimedate BIGINT,
mentiontimedate BIGINT,
mentiontype INT,
mentionsourcename varchar, -- note this is a varchar although http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf indicates it is an INT
mentionidentifier varchar, -- note this is a varchar although http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf indicates it is an INT
sentenceid INT,
actor1charoffset INT,
actor2charoffset INT,
actioncharoffset INT,
inrawtext INT,
confidence INT,
mentiondoclen INT,
mentiondoctone FLOAT,
mentiondoctranslationinfo varchar,
extras varchar
) from s3
target 'uri_location s3://gdelt-open-data, uri_path "/v2/mentions/*.mentions.csv", fmt_field_separator "\\t", ignore_invalid_records 1';

create view v_mentions as select * from events;
create view image v_mentions;


select eventtimedate / 1000000 "DATE", count(*), avg(mentiondoctone) 
from v_mentions 
where mentionidentifier imatching 'brexit' 
group by 1 
order by 1 desc;

In [ ]:
# example from class lab
df1 = spark.sql("""SELECT c1.name AS name1, c2.name AS name2, sub.charId1, sub.charId2, sub.pubCount
FROM
(
  SELECT r1.charId AS charId1, r2.charId AS charId2, COUNT(r1.pubId, r2.pubId) AS pubCount
  FROM relationships AS r1
  CROSS JOIN relationships AS r2
  WHERE r1.charId < r2.charId
  AND r1.pubId=r2.pubId
  GROUP BY r1.charId, r2.charId
) AS sub
INNER JOIN characters c1 ON c1.charId=sub.charId1
INNER JOIN characters c2 ON c2.charId=sub.charId2
ORDER BY sub.pubCount DESC
LIMIT 10""").cache()

In [112]:
sc.stop()